# Create local topography maps from different DEM sources with OGGM

There are a number of datasets available [out-of-the box](https://rgitools.readthedocs.io/en/latest/dems.html) in OGGM. This Notebook will show you how to download the original sources and create the local glacier centered map.
It is also possible to use your own DEM data in OGGM.

## Set-up 

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import oggm
from oggm import cfg, utils, workflow, tasks, graphics
from oggm.core import gis
cfg.initialize(logging_level='WARNING')
cfg.PARAMS['border'] = 10

## RGI outlines 

We use the RGI outlines to identify the necessary DEM tiles. If you haven't downloaded the RGI files yet, this will also download them. Feel free to use your desired RGI-ID here, otherwise let's use the Hintereisferner glacier as an example. 

In [ ]:
entity = utils.get_rgi_glacier_entities(['RGI60-11.00897'])
entity.plot();

## Choose a DEM source (e.g. SRTM)

If not specifying anything, OGGM will use it's default settings, i.e. NASADEM for mid- and low-latitudes (60°S-60°N). However, this needs registration at [NASA Earthdata](https://urs.earthdata.nasa.gov/) (see [below](#register)). Here, 
we choose the **SRTM** source as example DEM (no registration necessary).

In [ ]:
# Let's make a working directory for this DEM 
cfg.PATHS['working_dir'] = utils.gettempdir('default', reset=True)
gdir = workflow.init_glacier_directories(entity)[0]
tasks.define_glacier_region(gdir, source='SRTM')
# if not yet installed, you have to install rasterio

You can access the (reprojected and interpolated) DEM file in the working directory:

In [ ]:
dem_path = gdir.get_filepath('dem')
dem_path

It is a geotiff file. [Xarray](http://xarray.pydata.org) can open them thanks to [rasterio](https://rasterio.readthedocs.io):

In [ ]:
da = xr.open_rasterio(dem_path)
f, ax = plt.subplots()
da.plot(cmap='terrain', ax=ax);
# Add the outlines
gdir.read_shapefile('outlines').plot(ax=ax, color='none', edgecolor='black');

The source of the DEM is documented in the directory itself:

In [ ]:
with open(gdir.get_filepath('dem_source'), 'r') as f:
    print(f.read())

**OGGM is neither the owner nor the distributer of these datasets! OGGM only provides tools to access it. It is your responsibility as the data user to read the individual usage requirements and cite and acknowledge the original data sources accordingly.**

## OGGM provided datasets

At the moment OGGM is able to download and process the following DEM sources:

In [ ]:
for src in utils.DEM_SOURCES:
    print('{:<10}: {}'.format(src, gis.DEM_SOURCE_INFO[src].split('\n')[0]))

<a id='register'></a>
The default DEM source for low and mid-latitudes (60°S-60°N), **NASADEM**, requires a user account to download data, so you need to register at [NASA Earthdata](https://urs.earthdata.nasa.gov/). There are other DEM sources where a registration is necessary; for **ASTGTMV3** at [NASA Earthdata](https://urs.earthdata.nasa.gov/), for **TanDEM-X** at [DLR](https://sso.eoc.dlr.de/tdm90/selfservice/), and for **COPDEM** at [spacedata.copernicus.eu/](https://spacedata.copernicus.eu).

After that you can use the command line functionality `oggm_netrc_credentials` to store your user credentials in a local `~/.netrc` file. Your user credentials are only stored locally and are only used by the download function for authentification with the original DEM source. **Credentials are not needed if you use the RGI-TOPO data (see below).**

## Use pre-processed DEMs from RGI-TOPO 

The [RGI-TOPO](https://rgitools.readthedocs.io/en/latest/dems.html) dataset is an RGI-provided dataset in beta release. These data are available for everyone, and were created with OGGM. Of course you can easily use these data in OGGM as well:

In [ ]:
# use NASADEM, the default DEM for low and mid-latitudes in OGGM, you can also change to e.g. 'COPDEM'
from oggm.shop import rgitopo
cfg.PATHS['working_dir'] = utils.gettempdir('rgitopo', reset=True)
gdir = rgitopo.init_glacier_directories_from_rgitopo(['RGI60-11.00897'], dem_source='NASADEM')[0]
graphics.plot_domain(gdir)

## Use another DEM source

Using RGI-TOPO DEMs is by far the easiest since all data is prepared for you and ready to use. But if you really want, you can go back to the original data sources:

In [ ]:
# Let's make a working directory for this DEM 
cfg.PATHS['working_dir'] = utils.gettempdir('alternative')
try:
    gdir = workflow.init_glacier_directories(entity)[0]
    tasks.define_glacier_region(gdir, source='DEM3')
except oggm.exceptions.InvalidDEMError as err:
    print(err)

Let's check that the source text is updated as well:

In [ ]:
with open(gdir.get_filepath('dem_source'), 'r') as f:
    print(f.read())

In [ ]:
f, ax = plt.subplots()
da_dem3 = xr.open_rasterio(gdir.get_filepath('dem'))
da_dem3.plot(cmap='terrain', ax=ax);
gdir.read_shapefile('outlines').plot(ax=ax, color='none', edgecolor='black');

There might not be much difference a first sight, but by subtracting them the difference become clear:  

In [ ]:
f, ax = plt.subplots()
(da_dem3 - da).plot(ax=ax);
plt.title('DEM3 - SRTM');
gdir.read_shapefile('outlines').plot(ax=ax, color='none', edgecolor='black');

## Regional DEMs / DEM availability

Of course not all sources are available for every glacier as some of the DEMs are regional only. If we for example try the GIMP DEM, which is a Greenland specific DEM, it will not work for glaciers outside that region:

In [ ]:
# Let's make a working directory for this DEM 
cfg.PATHS['working_dir'] = utils.gettempdir('gimp', reset=True)
try:
    gdir = workflow.init_glacier_directories(entity)[0]
    tasks.define_glacier_region(gdir, source='GIMP')
except oggm.exceptions.InvalidDEMError as err:
    print(err)

## User provided DEM 

Users should be able to use any DEM file which can be opened by rasterio (i.e. geotiff). Here, we use a subset SRTM file shipped with OGGM as an example:

In [ ]:
custom_dem_path = utils.get_demo_file('hef_srtm.tif')
custom_dem_path

We tell OGGM to use it by changing the entry in the RGI table and by giving the path to the file:

In [ ]:
cfg.PATHS['dem_file'] = custom_dem_path

In [ ]:
cfg.PATHS['working_dir'] = utils.gettempdir('user', reset=True)
gdir = workflow.init_glacier_directories(entity)[0]
tasks.define_glacier_region(gdir, source='USER')

Now the user provided DEM is used:

In [ ]:
f, ax = plt.subplots()
da_user = xr.open_rasterio(gdir.get_filepath('dem'))
da_user.plot(cmap='terrain', ax=ax);
gdir.read_shapefile('outlines').plot(ax=ax, color='none', edgecolor='black');

## The border value, or how to chose the size of the topographic map

It is possible to specify the extent of the local topographic map. All maps are centered on the glacier and the size of the map is determined in grid points around the glacier. The number of grid points that was used in this example are 10 in order to save storage. But depending on your study you might need a larger topographic map. 

OGGM's [pre-processed directories](https://docs.oggm.org/en/stable/input-data.html#pre-processed-directories) come in 4 border sizes: 10, 40, 80 and 160. But if you process the topography yourself you can chose every value.

In [ ]:
# print the currently used number of gridpoints around a glacier
cfg.PARAMS['border']

In [ ]:
cfg.PARAMS['border'] = 1

In [ ]:
# Let's make a working directory for this DEM 
cfg.PATHS['working_dir'] = utils.gettempdir('border1')
gdir = workflow.init_glacier_directories(entity)[0]
tasks.define_glacier_region(gdir)
da = xr.open_rasterio(gdir.get_filepath('dem'))
f, ax = plt.subplots()
da.plot(cmap='terrain', ax=ax);
# Add the outlines
gdir.read_shapefile('outlines').plot(ax=ax, color='none', edgecolor='black');

In [ ]:
cfg.PARAMS['border'] = 100

In [ ]:
# Let's make a working directory for this DEM 
cfg.PATHS['working_dir'] = utils.gettempdir('border100')
gdir = workflow.init_glacier_directories(entity)[0]
tasks.define_glacier_region(gdir)
da = xr.open_rasterio(gdir.get_filepath('dem'))
f, ax = plt.subplots()
da.plot(cmap='terrain', ax=ax);
# Add the outlines
gdir.read_shapefile('outlines').plot(ax=ax, color='none', edgecolor='black');

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)